# Learning Manim myself by animating the Pythagoras Theorem

#### Install the Manim library

In [1]:
!pip install manim

In [2]:
from manim import *
#print(dir())

<code>dir()</code>: This built-in function returns a list of names in the current local scope or, if called with an object as an argument, it returns the names of the attributes and methods of that object.

#### Creating my First Scene
In Manim, animations are created by defining scenes within a class. Each scene is essentially a series of animations you want to play. Here’s how to create a simple scene with a circle.

#### Pythagoras Theorem Proof

In [ ]:
%%manim -ql -p PythagoreanTheorem 
import numpy as np
from manim import *

class PythagoreanTheorem(Scene):
    def construct(self):
         # Create the triangle with vertices at scaled coordinates
        scale_factor = 0.5  # Adjust this value to change the size of the triangle
        A = np.array([0, 0, 0]) * scale_factor  # Vertex A (0, 0)
        B = np.array([0, 4, 0]) * scale_factor  # Vertex B (0, 4)
        C = np.array([3, 0, 0]) * scale_factor  # Vertex C (3, 0)

        triangle = Polygon(A, B, C, fill_color=BLUE, fill_opacity=0.5)
        triangle.shift(LEFT * 1.5, DOWN * 1)

        # Calculate the side lengths for the squares
        side_AB_length = np.linalg.norm(B - A)
        side_BC_length = np.linalg.norm(C - B)
        side_CA_length = np.linalg.norm(A - C)

        # Create squares aligned to each triangle side
        square_AB = Square(side_length=side_AB_length, fill_color=YELLOW, fill_opacity=0.5)
        square_AB.next_to(triangle, LEFT, buff=0)
        square_AB.rotate(Line(A, B).get_angle())

        square_BC = Square(side_length=side_BC_length, fill_color=GREEN, fill_opacity=0.5)
        square_BC.next_to(triangle, RIGHT, buff=0)
        square_BC.rotate(Line(B, C).get_angle())
        square_BC.shift(UP * 1.2, LEFT * 1.1, DOWN * 0.43, RIGHT * 0.123)

        square_CA = Square(side_length=side_CA_length, fill_color=RED, fill_opacity=0.5)
        square_CA.next_to(triangle, DOWN, buff=0)
        square_CA.rotate(Line(C, A).get_angle())

        # Create text labels using MathTex for LaTeX rendering
        label_a2 = MathTex("a^2").move_to(square_CA.get_center())
        label_b2 = MathTex("b^2").move_to(square_AB.get_center())
        label_c2 = MathTex("c^2").move_to(square_BC.get_center())
        theorem_text = Text("Pythagoras Theorem").to_edge(UP)
        
        #equation_text = MathTex("a^2 + b^2 = c^2").next_to(square1, DOWN)

        self.play(Write(theorem_text))
        # Add triangle and squares to the scene
        self.play(Create(triangle))
        self.play(Create(square_AB),Create(square_BC),Create(square_CA))
        # Play the text animations
        self.play(Write(label_a2), Write(label_b2), Write(label_c2))
        #self.play(Write(equation_text))

        self.wait(2)
        
        # Function to create a gridded square
        def create_gridded_square(center, side_length, divisions, color):
            square_group = VGroup()
            small_square_length = side_length / divisions
            for i in range(divisions):
                for j in range(divisions):
                    small_square = Square(side_length=small_square_length, fill_color=color, fill_opacity=0.5)
                    # Position each small square within the grid
                    small_square.move_to(center + np.array([
                        (i - divisions / 2 + 0.5) * small_square_length,
                        (j - divisions / 2 + 0.5) * small_square_length,
                        0
                    ]))
                    square_group.add(small_square)
            return square_group

        # Create and align gridded squares for each triangle side
        grid_square_AB = create_gridded_square(
            center=square_AB.get_center(),
            side_length=side_AB_length,
            divisions=4,
            color=YELLOW
        )
        grid_square_AB.rotate(Line(A, B).get_angle())

        grid_square_BC = create_gridded_square(
            center=square_BC.get_center(),
            side_length=side_BC_length,
            divisions=5,
            color=GREEN
        )
        grid_square_BC.rotate(Line(B, C).get_angle())

        grid_square_CA = create_gridded_square(
            center=square_CA.get_center(),
            side_length=side_CA_length,
            divisions=3,
            color=RED
        )
        grid_square_CA.rotate(Line(C, A).get_angle())

        # Create plus and minus
        plus = Text("+").move_to(((grid_square_AB.get_center() + grid_square_CA.get_center()) / 2) + DOWN * 1.2)
        equal = Text("=").move_to((grid_square_BC.get_center() + grid_square_CA.get_center()) / 2 + DOWN * 1.6 + RIGHT*1.6)
            
        # Fade out the triangle
        self.play(FadeOut(triangle))
        self.play(Create(grid_square_AB), Create(grid_square_BC), Create(grid_square_CA))
        self.play(FadeOut(square_AB),FadeOut(square_BC),FadeOut(square_CA))
        
        # Animate moving the squares to their new positions
        self.play(grid_square_AB.animate.move_to(ORIGIN), grid_square_BC.animate.rotate(np.radians(52)).move_to(RIGHT * 3),  # Move to the right side
        grid_square_CA.animate.move_to(LEFT * 3))# Move to the left side
        self.play(label_b2.animate.move_to(grid_square_AB.get_center() + DOWN * 2), label_c2.animate.move_to(grid_square_BC.get_center() + DOWN * 2), label_a2.animate.move_to(grid_square_CA.get_center() + DOWN * 2))
        
        self.wait(1.5)
         # Move each small square from the 3x3 grid to fill the top-left 3x3 area of the 5x5 grid
        for i in range(3):
            for j in range(3):
                target_position = grid_square_BC[(4 - i) * 5 + j].get_center()  # Get the target position in the 5x5 grid
                self.play(grid_square_CA[i * 3 + j].animate.move_to(target_position), run_time=0.2)

         # Move the remaining squares from the 4x4 grid to fill the rest of the 5x5 grid
        remaining_positions = [
            grid_square_BC[i].get_center()
            for i in range(25)
            if i not in [(4 - i) * 5 + j for i in range(3) for j in range(3)]
        ]
        
        for i in range(16):
            self.play(grid_square_AB[i].animate.move_to(remaining_positions[i]), run_time=0.2)

        self.play(Write(plus),Write(equal))
        self.wait(2)


Manim Community v0.18.1

[10/28/24 09:54:43] INFO     Animation 0 : Using cached data (hash :                           ]8;id=841531;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=707914;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\renderer\cairo_renderer.py#88\88]8;;\
                             1185818338_3978709252_223132457)                                                      

[10/28/24 09:54:44] INFO     Animation 1 : Using cached data (hash :                           ]8;id=410080;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=843044;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\renderer\cairo_renderer.py#88\88]8;;\
                             624642324_3132203648_3775227054)                                                      

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=987987;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=412129;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\renderer\cairo_renderer.py#88\88]8;;\
                             624642324_276253285_1882451530)                                                       

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=846849;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=406757;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\renderer\cairo_renderer.py#88\88]8;;\
                             624642324_2247038272_1668889759)                                                      

...

[10/28/24 09:54:57] INFO     Animation 27 : Partial movie file written in                  ]8;id=552062;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=267443;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'C:\Users\anoop\OneDrive\Desktop\Simple                                               
                             Projects\manim_pythagoras_theorem\media\videos\manim_pythagor                         
                             as_theorem\480p15\partial_movie_files\PythagoreanTheorem\6246                         
                             42324_4173876313_2970594801.mp4'                                                      

[10/28/24 09:54:58] INFO     Animation 28 : Partial movie file written in                  ]8;id=503843;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=721010;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'C:\Users\anoop\OneDrive\Desktop\Simple                                               
                             Projects\manim_pythagoras_theorem\media\videos\manim_pythagor                         
                             as_theorem\480p15\partial_movie_files\PythagoreanTheorem\6246                         
                             42324_1040606922_20986479.mp4'                                                        

                    INFO     Animation 29 : Partial movie file written in                  ]8;id=609010;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=390426;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'C:\Users\anoop\OneDrive\Desktop\Simple                                               
                             Projects\manim_pythagoras_theorem\media\videos\manim_pythagor                         
                             as_theorem\480p15\partial_movie_files\PythagoreanTheorem\6246                         
                             42324_2090601257_3153581724.mp4'                                                      

[10/28/24 09:54:59] INFO     Animation 30 : Partial movie file written in                  ]8;id=230911;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=810366;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'C:\Users\anoop\OneDrive\Desktop\Simple                                               
                             Projects\manim_pythagoras_theorem\media\videos\manim_pythagor                         
                             as_theorem\480p15\partial_movie_files\PythagoreanTheorem\6246                         
                             42324_2980163638_3977105827.mp4'                                                      

                    INFO     Animation 31 : Partial movie file written in                  ]8;id=533329;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=812956;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'C:\Users\anoop\OneDrive\Desktop\Simple                                               
                             Projects\manim_pythagoras_theorem\media\videos\manim_pythagor                         
                             as_theorem\480p15\partial_movie_files\PythagoreanTheorem\6246                         
                             42324_917822060_2667676110.mp4'                                                       

The first line `%%manim -ql -p PythagoreanTheorem` is a magic command used in Jupyter notebooks to run a Manim scene directly from a notebook cell. Here’s a breakdown of what each part means:

### Breakdown of the Command:
- **`%%manim`**: This is a cell magic command specific to Jupyter notebooks that allows you to execute Manim code within the cell. When you run this cell, it tells Jupyter to process the contents using the Manim library.

- **`-ql`**: This stands for "quality low". It sets the rendering quality to low for quicker rendering times during development. In Manim, you can specify different quality settings:
  - **`-ql`**: Low quality 
  - **`-qm`**: Medium quality 
  - **`-qh`**: High quality 

- **`-p`**: This flag tells Manim to automatically play the rendered video once it's created, opening the video file in your default media player.

- **`PythagoreanTheorem`**: This is the name of the scene (class) to render.